Import Packages

In [8]:
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import  KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score,roc_curve,auc 
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score, classification_report, f1_score, average_precision_score, precision_recall_fscore_support)
import seaborn as sns
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import statistics
from sklearn.model_selection import RandomizedSearchCV
from scipy import interp
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score,roc_curve,auc 
from sklearn.metrics import recall_score, precision_score

In [1]:
raw_data = pd.read_csv('tranformLuric.csv')

Inspect data 

In [14]:
def univariate(df,col,vartype,hue =None):    
    '''
    Univariate function will plot parameter values in graphs.
    df      : dataframe name
    col     : Column name
    vartype : variable type : continuous or categorical
                Continuous(0)   : Distribution, Violin & Boxplot will be plotted.
                Categorical(1) : Countplot will be plotted.
    hue     : Only applicable in categorical analysis.
    '''
    sns.set(style="darkgrid")
    if vartype == 0:
        fig, ax=plt.subplots(nrows =1,ncols=3,figsize=(20,8))
        ax[0].set_title("Distribution Plot")
        sns.distplot(df[col],ax=ax[0])
        ax[1].set_title("Violin Plot")
        sns.violinplot(data =df, x=col,ax=ax[1], inner="quartile")
        ax[2].set_title("Box Plot")
        sns.boxplot(data =df, x=col,ax=ax[2],orient='v')
    if vartype == 1:
        temp = pd.Series(data = hue)
        fig, ax = plt.subplots()
        width = len(df[col].unique()) + 6 + 4*len(temp.unique())
        fig.set_size_inches(width , 7)
        ax = sns.countplot(data = df, x= col, order=df[col].value_counts().index,hue = hue) 
        if len(temp.unique()) > 0:
            for p in ax.patches:
                ax.annotate('{:1.1f}%'.format((p.get_height()*100)/float(len(df))), (p.get_x()+0.05, p.get_height()+20))  
        else:
            for p in ax.patches:
                ax.annotate(p.get_height(), (p.get_x()+0.32, p.get_height()+20)) 
        del temp
    else:
        exit
    plt.show()
    
#univariate(df=raw_data,col='cadyn',vartype=1)

Helper Functions

In [15]:

def apply_mean_center(x):
    mu = np.mean(x)
    xm = x/mu
    return xm, mu

def mean_center_transform(df, cols):
    df_new = pd.DataFrame()
    sc = {}
    for col in cols:
        x = df[col].values
        df_new[col], mu = apply_mean_center(x)
        sc[col] = mu
    return df_new, sc

def apply_standardize(x):
    mu = np.mean(x)
    sd = np.std(x)
    xm = (x-mu)/sd
    return xm, mu, sd

def standardize_transform(df, cols):
    df_new = pd.DataFrame()
    mean = {}
    sd = {}
    for col in cols:
        x = df[col].values
        df_new[col], mu, std = apply_standardize(x)
        mean[col] = mu
        sd[col] = std
    return df_new, mean, sd

def apply_scale(x):
    x_min = np.min(x)
    x_max = np.max(x)
    xm = (x-x_min)/(x_max-x_min)
    return xm, x_min, x_max

def scale_transform(df, cols):
    df_new = pd.DataFrame()
    min_dict = {}
    max_dict = {}
    for col in cols:
        x = df[col].values
        df_new[col], x_min, x_max = apply_scale(x)
        min_dict[col] = x_min
        max_dict[col] = x_max
    return df_new, min_dict, max_dict
def float_to_int (df):
    float_col = df.select_dtypes(include=('string'))
    for col in float_col.columns.values:
         raw_data[col] = raw_data[col].astype('float64')
    return raw_data
raw_data = float_to_int(raw_data)
for col in raw_data.columns:
      raw_data[col] = pd.to_numeric(raw_data[col],errors = 'coerce')

Outliers

In [35]:
def detect_outliers(df, threshold=3):
    """
    Replace outliers with NaN values for each column separately
    """
    df_out = pd.DataFrame()
    for col in df.columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - threshold*iqr
        upper_bound = q3 + threshold*iqr
        df_out[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), np.nan, df[col])
    return df_out


data_without_outliers = detect_outliers_zscore(raw_data)

def detect_outliers_zscore(df, threshold=3):
    """
    Replace outliers with NaN values for each column separately using Z-score approach
    """
    df_out = pd.DataFrame()
    for col in df.columns:
        zscores = (df[col] - df[col].mean()) / df[col].std()
        df_out[col] = np.where(np.abs(zscores) > threshold, np.nan, df[col])
    return df_out


data_without_outliers_zscore = detect_outliers_zscore(raw_data)

from sklearn.neighbors import LocalOutlierFactor

def detect_outliers_lof(df, n_neighbors=20, contamination=0.1):
    """
    Replace outliers with NaN values using LOF approach for each column separately
    """
    df_out = pd.DataFrame()
    for col in df.columns:
        clf = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
        outlier_scores = clf.fit_predict(df[col].values.reshape(-1, 1))
        df_out[col] = np.where(outlier_scores == -1, np.nan, df[col])
    return df_out

data_without_outliers_LOF = detect_outliers_lof(raw_data)
# Print the number of missing values in each column
#print(data_without_outliers_LOF.isna().sum())

Missingness

In [ ]:
def missing_values(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
    
    # Drop columns with >= 25 % of missing values 
    to_keep = (missing_value_df[missing_value_df.iloc[:,-1]<=25.0].index)
    df_new = df.drop(df.columns.difference(to_keep), axis=1)

    return [missing_value_df, df_new]

def impute_mixed_var_new(df_data, rv_type):
    imp_knn = KNNImputer(n_neighbors=2)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_med = SimpleImputer(missing_values=np.nan, strategy='median')
    imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    for rvs in list(df_data):
        data_impute = imp_mean.fit_transform(pd.DataFrame(df_data[rvs]))
        data_impute = imp_knn.fit_transform(pd.DataFrame(df_data[rvs]))
        if rv_type[rvs] == 'g':
            # data_impute = imp_knn.fit_transform(pd.DataFrame(df_data[rvs]))
            data_impute = imp_mean.fit_transform(pd.DataFrame(df_data[rvs]))
        elif rv_type[rvs] == 'p':
            data_impute = imp_med.fit_transform(pd.DataFrame(df_data[rvs]))
        else:
            data_impute = imp_freq.fit_transform(pd.DataFrame(df_data[rvs]))
        df_data[rvs] = data_impute
    return df_data

Import data

In [ ]:
# Extract the feature matrix X and the target vector y
X = df.drop('target', axis=1).values
y = df['target'].values

Feature Selection (Include results from R code)

In [ ]:
from scipy.stats import pearsonr
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from skrebate import ReliefF

# Compute the correlation coefficient and p-value between each feature and the target
correlations, p_values = [], []
for feature in range(X.shape[1]):
    corr, p_val = pearsonr(X[:, feature], y)
    correlations.append(abs(corr))
    p_values.append(p_val)

# Select the top K features with the highest correlation coefficient
K = 5
selected_features = df.drop('target', axis=1).columns[sorted(range(X.shape[1]), key=lambda i: correlations[i], reverse=True)[:K]]

# Subset the data with the selected features
X_new = df[selected_features].values

# Compute the mutual information between each feature and the target
mi = mutual_info_classif(X, y)

# Select the top K features with the highest mutual information
K = 5
selector = SelectKBest(mutual_info_classif, k=K)
X_new = selector.fit_transform(X, y)

# Compute the feature importance using ReliefF
estimator = KNeighborsClassifier(n_neighbors=5)
fs = ReliefF(n_features_to_select=K, n_neighbors=100, estimator=estimator)
fs.fit(X, y)
importances = fs.feature_importances_

# Select the top K features with the highest feature importance
selected_features = df.drop('target', axis=1).columns[importances.argsort()[::-1][:K]]
X_new = df[selected_features].values

#######################################################################################
def backward_elimination(data, target,significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant.astype(float)).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features

Multicollinearity

In [ ]:
import pandas as pd
import seaborn as sns
# Create correlation matrix
corr = data.corr()
# Plot heatmap and Remove one of the correlated variables: If two or more variables are highly correlated, we can remove one of them to reduce the multicollinearity. 
sns.heatmap(corr, annot=True, cmap='coolwarm')
#############################################################################################
#VIF (Variance Inflation Factor) is another commonly used method for detecting multicollinearity
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Drop the target variable and categorical variables, if any
X = data.drop(['target', 'categorical_var'], axis=1)

# Calculate VIF
vif = pd.DataFrame()
vif["features"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)

###########################################################################################
#Use regularization: Regularization techniques, such as ridge regression 
#and lasso regression, can be used to reduce the impact of multicollinearity. 
#These techniques add a penalty term to the regression equation, 
#which helps to reduce the coefficients of highly correlated variables.

from sklearn.linear_model import Ridge

# Create Ridge regression object
ridge = Ridge(alpha=0.5)

# Fit model
ridge.fit(X, y)



Model 

In [ ]:
#Import raw data

cols = ['age','sex','eapoa1','Troponin_T','smoclass','elpa','ace','haptoglo','cadyn']
data = raw_data[cols]

x_new= data.drop(['cadyn'],axis=1)
y_new= data.iloc[:,-1] 


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold

class Metrics:
    def __init__(self, y_test, y_score, yhat):
        self.y_test = y_test
        self.y_score = y_score
        self.yhat = yhat
        
    def accuracy(self):
        return accuracy_score(self.y_test, self.y_score)
    
    def sensitivity(self):
        return recall_score(self.y_test, self.y_score, pos_label=1)
    
    def specificity(self):
        tn, fp, fn, tp = confusion_matrix(self.y_test, self.y_score).ravel()
        return tn / (tn + fp)
    
    def roc_auc(self):
        fpr, tpr, thresholds = roc_curve(self.y_test, self.yhat)
        return auc(fpr, tpr)
    
    def recall(self):
        return recall_score(self.y_test, self.y_score, average='macro')
    
    def precision(self):
        return precision_score(self.y_test, self.y_score, zero_division=0)

def run_classification(model, x_new, y_new):
    accs, senss, specs, tprs, aucs = [], [], [], [], []
    cms = np.zeros((2, 2)) # initialize empty confusion matrix
    cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    for train_index, test_index in cv_outer.split(x_new, y_new):
        x_train, x_test = x_new.iloc[train_index], x_new.iloc[test_index]
        y_train, y_test = y_new.iloc[train_index], y_new.iloc[test_index]
        model.fit(x_train, y_train)
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(x_test)
        expected_value = explainer.expected_value
        yhat = model.predict_proba(x_test)[:, 1]
        y_score = (yhat > 0.5) * 1
        metrics = Metrics(y_test, y_score, yhat)
        accs.append(metrics.accuracy())
        senss.append(metrics.sensitivity())
        specs.append(metrics.specificity())
        tprs.append(np.interp(mean_fpr, metrics.fpr(), metrics.tpr()))
        tprs[-1][0] = 0.0
        roc_auc = metrics.roc_auc()
        aucs.append(roc_auc)
        cm = confusion_matrix(y_test, y_score)
        cms += cm
        # Generate summary plot for each fold
        shap.summary_plot(shap_values, x_test, title="SHAP summary plot for fold")
    avg_cm = cms / 10 # calculate average confusion matrix
    return (np.mean(accs), np.mean(senss), np.mean(specs), np.mean(aucs), np.mean(tprs, axis=0), avg_cm)


# Define the models to be tested
classifiers = [
    RandomForestClassifier(bootstrap=True, max_depth=100, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=100),
    GradientBoostingClassifier(criterion='friedman_mse', loss='exponential', min_samples_leaf=3, min_samples_split=2, n_estimators=150),
]

# Perform classification and store results
results = []
for model in classifiers:
    print("Model:", model)
    results.append(run_classification(model, x_new, y_new))

# Display results
for i, model in enumerate(classifiers):
    print(type(model).__name__)
    print('Accuracy:', results[i][0])
    print('Sensitivity:', results[i][1])
    print('Specificity:', results[i][2])
    print('AUC:', results[i][3])
    print('Average Confusion Matrix:')
    print(results[i][5])
    print('\n')
    
    
# Generate summary bar plot for each model
for i, model in enumerate(classifiers):
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(x_new)
    expected_value = explainer.expected_value
    shap.summary_plot(shap_values, x_new, plot_type="bar", title="SHAP summary plot for {}".format(type(model).__name__))

    
# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--') 
for i, model in enumerate(classifiers):
    mean_tprs = results[i][4]
    mean_tprs[-1] = 1.
    mean_fpr = np.linspace(0, 1, 100)
    plt.plot(mean_fpr, mean_tprs, label='{} (AUC = {:.2f})'.format(type(model).__name__, results[i][3]))
    
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc='lower right')
plt.show()
